# Baseband and Scintillometry Walk Through

Welcome to the Scintillometry package walk through notebook. Here you will learn the knowledge of pulsar and the basic usage of baseband and scintillometry python packages.

## Outline
* Introducation
* Read baseband data using baseband pacakge
* Dedispersion
* Integration
    * Waterfall plot
    * Folding
    * Polarizations 
* Output

## Introduction

### Baseband data

### Pulsars 

### Pulsar data

### Pulsar data process

#### Required python package
1. [Baseband](https://github.com/mhvk/baseband) 
2. [Numpy](http://www.numpy.org/) 
3. [Astropy](http://www.astropy.org/) 
4. [Scintillometry](https://github.com/mhvk/scintillometry) 
#### Optional python package 
5. [PINT](https://github.com/nanograv/PINT) 

## Read baseband data

1. Decide your data types
    * Supported data types
        * [VDIF](https://baseband.readthedocs.io/en/stable/vdif/index.html)
        * [MARK 5B](https://baseband.readthedocs.io/en/stable/mark5b/index.html)
        * [MARK 4](https://baseband.readthedocs.io/en/stable/mark4/index.html)
        * [DADA](https://baseband.readthedocs.io/en/stable/dada/index.html)
        * [GUPPI](https://baseband.readthedocs.io/en/stable/guppi/index.html)
        * [GSB](https://baseband.readthedocs.io/en/stable/gsb/index.html)


Note: There is a detailed example code in the link to each file types.

2. Import the necessary module. For instance, we use GUPPI file type

In [2]:
import numpy as np  
import astropy.units as u  # import astropy units module, this helps you on the physical quantity unit convers
from baseband import guppi # import the guppi baseband reader 
import os # This package will help you navigating the file system. 

3. Create the file handler for your files. 
<br>3.1 Read a single file.

In [5]:
# I think the baseband part should go to the baseband package.
data_dir = '/home/luo/Projects/baseband/baseband/data'
data_file = os.path.join(data_dir, 'sample_puppi.raw')
fh = guppi.open(data_file, 'rb') # 'rb' means read binary file, since the data file is in binary format. 

In [12]:
fh.read_header()

UnicodeDecodeError: 'ascii' codec can't decode byte 0xf6 in position 0: ordinal not in range(128)

<br>3.2 Read mulitple files together.  
Note: Those files should be contiguous and sorted by time.  

4. Read and exam the header information

In [11]:
fh.

GUPPIFileReader(fh_raw=<_io.BufferedReader name='/home/luo/Projects/baseband/baseband/data/sample_puppi.raw'>)

In [ ]:
"""Test of the scintillometry dedispersion routines on PSR B1937+21"""
import numpy as np
import astropy.units as u
from baseband import guppi
from scintillometry.dispersion import Dedisperse
from scintillometry.functions import Square
from scintillometry.fourier import get_fft_maker
import matplotlib.pylab as plt


FFT = get_fft_maker('pyfftw', flags=['FFTW_ESTIMATE'], threads=4)

dm = 71.02227638 * u.pc / u.cm**3
f0 = (641.928224582360 + 2.78557046583437451/60.) * u.Hz  # from polyco

fh = guppi.open('puppi_58245_B1937+21_0799.0010.raw')
frequency = (fh.header0['OBSFREQ']
             - fh.header0['OBSBW'] / 2.  # Down to bottom end
             + fh.header0['CHAN_BW'] / 2.  # Up to center of channel 0
             + fh.header0['CHAN_BW'] * np.arange(4)) * u.MHz

dedisperse = Dedisperse(fh, dm, frequency=frequency, sideband=1,
                        FFT=FFT)

square = Square(dedisperse)

nbin = 64
lc = np.zeros(nbin)
for i in range(10):
    print('At ', square.tell(unit=u.s))
    pre = square.tell()
    p = square.read(3125000).sum((1, 2))
    post = square.tell()
    ph = (np.arange(pre, post) / fh.sample_rate * f0).to_value(u.one)
    ph = (ph - 0.35) % 1.
    lc += np.bincount((ph * nbin).astype(int), weights=p)

plt.ion()
plt.plot(np.arange(nbin)/nbin, lc)

Useful links: <br>
&nbsp;&nbsp;&nbsp;&nbsp;Baseband documentation: https://baseband.readthedocs.io/en/stable/ 
<br>
&nbsp;&nbsp;&nbsp;&nbsp;Baseband github: https://github.com/mhvk/baseband

## Dedispersion

## Integration

## Output